In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

zip_path = '/content/drive/My Drive/output.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
import zipfile

zip_path = '/content/drive/My Drive/data/output.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('data2')

In [ ]:
import numpy as np
import cv2
import os
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.utils.image_utils import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tensorflow.keras.optimizers import SGD
import argparse

In [ ]:
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        # Lưu ảnh tiền xử lý
        self.preprocessors = preprocessors

        # Nếu bước tiền xử lý là None thì khởi tạo danh sách rỗng
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, imagePaths, verbose=-1):
        # Khởi tạo danh sách các đặc trưng và nhãn
        data = []
        labels = []

        # Lặp qua tất cả ảnh đầu vào
        for (i, imagePath) in enumerate(imagePaths):
            # Nạp ảnh và trích xuất nhãn từ đường dẫn định dạng
            # /path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            # check to see if our preprocessors are not None
            if self.preprocessors is not None:
                # Lặp qua tất cả tiền xử lý và áp dụng cho mỗi ảnh
                for p in self.preprocessors:
                    image = p.preprocess(image)
            # Mỗi ảnh được xử lý là vector đặc trưng bằng cách
            # cập nhật danh sách dữ liệu cùng với nhãn
            data.append(image)
            labels.append(label)

            # Hiển thị ảnh cập nhật
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
               print("[INFO] Đã xử lý {}/{}".format(i + 1,len(imagePaths)))
                # Trả về dữ liệu kiểu tuple gồm dữ liệu và nhãn
        return (np.array(data), np.array(labels))

In [ ]:
class ImageToArrayPreprocessor:  # Tạo lớp để chuyển ảnh --> mảng
    def __init__(self, dataFormat=None):
        # Lưu ảnh đã được định dạng
        self.dataFormat = dataFormat

    def preprocess(self, image): # Định nghĩa phương thức preprocess trả về mảng
        # Hàm img_to_array của Keras
        return img_to_array(image, data_format=self.dataFormat)
class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # Lưu image width, height và interpolation
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        # Trả về ảnh có kích thước đã thay đổi
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

In [ ]:
class MiniVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # Khởi tạo mô hình, shape ảnh đầu vào và số kênh của ảnh đầu vào
        model = Sequential()
        input_shape = (height, width, depth)
        channel_dim = -1  # chỉ số của số kênh ảnh đầu vào
                          # giá trị -1 ý muốn nói chỉ số kênh nằm cuối cùng
                          # của danh sách chứa dữ liệu ảnh đầu vào

        # sử dụng 'channels_first' để cập nhật shape và số kênh ảnh đầu vào
        if K.image_data_format() == 'channels_first':
            input_shape = (depth, height, width)
            channel_dim = 1

        # Chuỗi layer đầu tiên  CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(32, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Chuỗi layer thứ hai CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Thiết lập FC thứ nhất => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))   # Dropout 50%

        # THiết lập FC thứ hai => Hàm phân lớp Softmax
        model.add(Dense(classes))
        model.add(Activation('softmax'))

        # Trả về kiến trúc mạng/mô hình
        return model

code thêm

In [ ]:
from multiprocessing import Pool
def load_image(imagePath):
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (224, 224))
    return image
imagePaths = list(paths.list_images("data/output"))

    # Sử dụng multiprocessing để nạp ảnh đồng thời
with Pool() as p:
    data = p.map(load_image, imagePaths)

    # Chuyển đổi ảnh sang mảng và chuẩn bị dữ liệu
data = np.array(data) / 255.0
labels = [p.split(os.path.sep)[-2] for p in imagePaths]
labels = np.array(labels)

code gốc

In [ ]:
imagePaths = list(paths.list_images("data/output"))

# Bước 1. Chuẩn bị dữ liệu
# Khởi tạo tiền xử lý ảnh
sp = SimplePreprocessor(32, 32) # Thiết lập kích thước ảnh 32 x 32
iap = ImageToArrayPreprocessor() # Gọi hàm để chuyển ảnh sang mảng

# Nạp dataset từ đĩa
print("[INFO] Nạp ảnh...")

sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model =MiniVGGNet.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
305/305 [==============================] - 7s 14ms/step - loss: 0.9721 - accuracy: 0.5784 - val_loss: 2.0280 - val_accuracy: 0.3741
Epoch 2/50
305/305 [==============================] - 4s 13ms/step - loss: 0.8130 - accuracy: 0.6308 - val_loss: 1.3343 - val_accuracy: 0.4982
Epoch 3/50
305/305 [==============================] - 4s 13ms/step - loss: 0.7648 - accuracy: 0.6495 - val_loss: 0.8922 - val_accuracy: 0.6160
Epoch 4/50
305/305 [==============================] - 4s 12ms/step - loss: 0.6936 - accuracy: 0.6774 - val_loss: 0.6907 - val_accuracy: 0.6575
Epoch 5/50
305/305 [==============================] - 3s 11ms/step - loss: 0.6477 - accuracy: 0.6986 - val_loss: 0.6831 - val_accuracy: 0.6701
Epoch 6/50
305/305 [==============================] - 4s 12ms/step - loss: 0.6190 - accuracy: 0.7136 - val_loss: 0.6015 - val_accuracy: 0.7247
Epoch 7/50
305/305 [==============================] - 4s 13ms/step - loss: 0.5811 - acc

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = Adam(learning_rate=0.01)

model =MiniVGGNet.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
305/305 [==============================] - 5s 14ms/step - loss: 0.9767 - accuracy: 0.5861 - val_loss: 0.7944 - val_accuracy: 0.5946
Epoch 2/50
305/305 [==============================] - 4s 13ms/step - loss: 0.7110 - accuracy: 0.6586 - val_loss: 0.6275 - val_accuracy: 0.6943
Epoch 3/50
305/305 [==============================] - 3s 11ms/step - loss: 0.6204 - accuracy: 0.7084 - val_loss: 1.2698 - val_accuracy: 0.4911
Epoch 4/50
305/305 [==============================] - 4s 12ms/step - loss: 0.5651 - accuracy: 0.7405 - val_loss: 0.5392 - val_accuracy: 0.7462
Epoch 5/50
305/305 [==============================] - 4s 13ms/step - loss: 0.5279 - accuracy: 0.7631 - val_loss: 0.8317 - val_accuracy: 0.6268
Epoch 6/50
305/305 [==============================] - 4s 13ms/step - loss: 0.5053 - accuracy: 0.7713 - val_loss: 0.8873 - val_accuracy: 0.6626
Epoch 7/50
305/305 [==============================] - 4s 12ms/step - loss: 0.5022 - acc

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer =  Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7)

model =MiniVGGNet.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
305/305 [==============================] - 5s 14ms/step - loss: 0.9538 - accuracy: 0.6113 - val_loss: 8.8464 - val_accuracy: 0.3347
Epoch 2/50
305/305 [==============================] - 4s 13ms/step - loss: 0.6648 - accuracy: 0.6910 - val_loss: 1.3397 - val_accuracy: 0.4456
Epoch 3/50
305/305 [==============================] - 4s 12ms/step - loss: 0.5909 - accuracy: 0.7260 - val_loss: 0.6484 - val_accuracy: 0.6932
Epoch 4/50
305/305 [==============================] - 3s 11ms/step - loss: 0.5490 - accuracy: 0.7474 - val_loss: 0.7347 - val_accuracy: 0.6589
Epoch 5/50
305/305 [==============================] - 4s 12ms/step - loss: 0.5122 - accuracy: 0.7654 - val_loss: 0.6532 - val_accuracy: 0.7319
Epoch 6/50
305/305 [==============================] - 4s 12ms/step - loss: 0.4844 - accuracy: 0.7817 - val_loss: 0.5802 - val_accuracy: 0.7436
Epoch 7/50
305/305 [==============================] - 4s 12ms/step - loss: 0.4635 - acc

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer =  Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-7)

model =MiniVGGNet.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
305/305 [==============================] - 5s 13ms/step - loss: 0.9742 - accuracy: 0.5892 - val_loss: 1.2765 - val_accuracy: 0.4239
Epoch 2/50
305/305 [==============================] - 4s 13ms/step - loss: 0.7276 - accuracy: 0.6481 - val_loss: 1.1866 - val_accuracy: 0.5186
Epoch 3/50
305/305 [==============================] - 4s 12ms/step - loss: 0.6538 - accuracy: 0.6856 - val_loss: 0.8268 - val_accuracy: 0.6055
Epoch 4/50
305/305 [==============================] - 3s 11ms/step - loss: 0.5970 - accuracy: 0.7206 - val_loss: 0.8987 - val_accuracy: 0.6003
Epoch 5/50
305/305 [==============================] - 3s 11ms/step - loss: 0.5528 - accuracy: 0.7429 - val_loss: 0.6159 - val_accuracy: 0.6971
Epoch 6/50
305/305 [==============================] - 4s 13ms/step - loss: 0.5355 - accuracy: 0.7578 - val_loss: 0.5406 - val_accuracy: 0.7518
Epoch 7/50
305/305 [==============================] - 3s 11ms/step - loss: 0.5154 - acc

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer =  Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-7)

model =MiniVGGNet.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=32, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
609/609 [==============================] - 7s 9ms/step - loss: 1.0051 - accuracy: 0.5788 - val_loss: 0.7413 - val_accuracy: 0.6265
Epoch 2/50
609/609 [==============================] - 5s 8ms/step - loss: 0.7106 - accuracy: 0.6556 - val_loss: 0.8269 - val_accuracy: 0.6052
Epoch 3/50
609/609 [==============================] - 6s 9ms/step - loss: 0.6536 - accuracy: 0.6870 - val_loss: 0.8990 - val_accuracy: 0.5735
Epoch 4/50
609/609 [==============================] - 5s 8ms/step - loss: 0.6190 - accuracy: 0.7074 - val_loss: 0.6710 - val_accuracy: 0.6766
Epoch 5/50
609/609 [==============================] - 6s 9ms/step - loss: 0.6479 - accuracy: 0.6930 - val_loss: 0.7138 - val_accuracy: 0.6373
Epoch 6/50
609/609 [==============================] - 5s 8ms/step - loss: 0.6060 - accuracy: 0.7214 - val_loss: 0.5865 - val_accuracy: 0.7228
Epoch 7/50
609/609 [==============================] - 5s 8ms/step - loss: 0.5969 - accuracy: 

In [ ]:
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        # Khởi tạo mô hình, shape ảnh đầu vào và số kênh của ảnh đầu vào
        model = Sequential()
        inputShape = (height, width, depth)

        # sử dụng 'channels_first' để cập nhật shape và số kênh ảnh đầu vào
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)

        # thêm layer Convolutional, Activation (ReLU) và max-pooling
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # thêm layer Convolutional, Activation (ReLU) và max-pooling
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # flatten layer trước khi đưa vào Fully Connected (FC) layer
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        # output layer với softmax activation
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # trả về kiến trúc mô hình
        return model


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

def VGG16(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model

In [ ]:
imagePaths = list(paths.list_images("data/output"))

# Bước 1. Chuẩn bị dữ liệu
# Khởi tạo tiền xử lý ảnh
sp = SimplePreprocessor(32, 32) # Thiết lập kích thước ảnh 32 x 32
iap = ImageToArrayPreprocessor() # Gọi hàm để chuyển ảnh sang mảng

# Nạp dataset từ đĩa
print("[INFO] Nạp ảnh...")

sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

[INFO] Nạp ảnh...
[INFO] Đã xử lý 500/25958
[INFO] Đã xử lý 1000/25958
[INFO] Đã xử lý 1500/25958
[INFO] Đã xử lý 2000/25958
[INFO] Đã xử lý 2500/25958
[INFO] Đã xử lý 3000/25958
[INFO] Đã xử lý 3500/25958
[INFO] Đã xử lý 4000/25958
[INFO] Đã xử lý 4500/25958
[INFO] Đã xử lý 5000/25958
[INFO] Đã xử lý 5500/25958
[INFO] Đã xử lý 6000/25958
[INFO] Đã xử lý 6500/25958
[INFO] Đã xử lý 7000/25958
[INFO] Đã xử lý 7500/25958
[INFO] Đã xử lý 8000/25958
[INFO] Đã xử lý 8500/25958
[INFO] Đã xử lý 9000/25958
[INFO] Đã xử lý 9500/25958
[INFO] Đã xử lý 10000/25958
[INFO] Đã xử lý 10500/25958
[INFO] Đã xử lý 11000/25958
[INFO] Đã xử lý 11500/25958
[INFO] Đã xử lý 12000/25958
[INFO] Đã xử lý 12500/25958
[INFO] Đã xử lý 13000/25958
[INFO] Đã xử lý 13500/25958
[INFO] Đã xử lý 14000/25958
[INFO] Đã xử lý 14500/25958
[INFO] Đã xử lý 15000/25958
[INFO] Đã xử lý 15500/25958
[INFO] Đã xử lý 16000/25958
[INFO] Đã xử lý 16500/25958
[INFO] Đã xử lý 17000/25958
[INFO] Đã xử lý 17500/25958
[INFO] Đã xử lý 18000/

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

input_shape = (32, 32, 3)
num_classes = 3

model = VGG16(input_shape,num_classes)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
305/305 [==============================] - 25s 43ms/step - loss: 1.0990 - accuracy: 0.3343 - val_loss: 1.0986 - val_accuracy: 0.3347
Epoch 2/50
305/305 [==============================] - 12s 39ms/step - loss: 1.0969 - accuracy: 0.3453 - val_loss: 1.0796 - val_accuracy: 0.4488
Epoch 3/50
305/305 [==============================] - 12s 39ms/step - loss: 1.0142 - accuracy: 0.4601 - val_loss: 1.1154 - val_accuracy: 0.3347
Epoch 4/50
305/305 [==============================] - 12s 38ms/step - loss: 1.0516 - accuracy: 0.4074 - val_loss: 1.0227 - val_accuracy: 0.4670
Epoch 5/50
305/305 [==============================] - 12s 40ms/step - loss: 0.8631 - accuracy: 0.5426 - val_loss: 0.7645 - val_accuracy: 0.5867
Epoch 6/50
305/305 [==============================] - 12s 40ms/step - loss: 0.7529 - accuracy: 0.5873 - val_loss: 0.9197 - val_accuracy: 0.5522
Epoch 7/50
305/305 [==============================] - 12s 39ms/step - loss: 0.697

In [ ]:
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from keras.models import Model

def mini_resnet(input_shape, num_classes):
    # input layer
    x_in = Input(shape=input_shape)

    # stage 1
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x_in)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # stage 2
    x_shortcut = x
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x_shortcut, x])
    x = Activation('relu')(x)

    # stage 3
    x_shortcut = x
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    # shortcut path
    x_shortcut = Conv2D(filters=64, kernel_size=(1, 1), strides=(2, 2), padding='same')(x_shortcut)
    x_shortcut = BatchNormalization()(x_shortcut)

    x = Add()([x_shortcut, x])
    x = Activation('relu')(x)

    # stage 4
    x_shortcut = x
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    # shortcut path
    x_shortcut = Conv2D(filters=128, kernel_size=(1, 1), strides=(2, 2), padding='same')(x_shortcut)
    x_shortcut = BatchNormalization()(x_shortcut)

    x = Add()([x_shortcut, x])
    x = Activation('relu')(x)

    # final stage
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)

    # create model
    model = Model(inputs=x_in, outputs=x, name='mini_resnet')
    return model


In [ ]:
imagePaths = list(paths.list_images("data/output"))

# Bước 1. Chuẩn bị dữ liệu
# Khởi tạo tiền xử lý ảnh
sp = SimplePreprocessor(32, 32) # Thiết lập kích thước ảnh 32 x 32
iap = ImageToArrayPreprocessor() # Gọi hàm để chuyển ảnh sang mảng

# Nạp dataset từ đĩa
print("[INFO] Nạp ảnh...")

sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

input_shape = (32, 32, 3)
num_classes = 3

model = mini_resnet(input_shape,num_classes)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])


# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

In [ ]:
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense, LeakyReLU, Dropout
from keras.models import Model

def mini_resnet_v2(input_shape, num_classes):
    # input layer
    x_in = Input(shape=input_shape)

    # stage 1
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x_in)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    # stage 2
    x_shortcut = x
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    # Add a Conv2D layer to reduce the dimension of x_shortcut
    x_shortcut = Conv2D(filters=64, kernel_size=(1, 1), padding='same')(x_shortcut)
    x_shortcut = BatchNormalization()(x_shortcut)

    x = Add()([x_shortcut, x])
    x = LeakyReLU(alpha=0.1)(x)

    # stage 3
    x_shortcut = x
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    # shortcut path
    x_shortcut = Conv2D(filters=128, kernel_size=(1, 1), strides=(2, 2), padding='same')(x_shortcut)
    x_shortcut = BatchNormalization()(x_shortcut)

    x = Add()([x_shortcut, x])
    x = LeakyReLU(alpha=0.1)(x)

    # stage 4
    x_shortcut = x
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    # shortcut path
    x_shortcut = Conv2D(filters=256, kernel_size=(1, 1), strides=(2, 2), padding='same')(x_shortcut)
    x_shortcut = BatchNormalization()(x_shortcut)

    x = Add()([x_shortcut, x])
    x = LeakyReLU(alpha=0.1)(x)

    # global average pooling and output
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=512)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(rate=0.5)(x)
    x_out = Dense(units=num_classes, activation='softmax')(x)

    # model
    model = Model(inputs=x_in, outputs=x_out)

    return model


In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

input_shape = (32, 32, 3)
num_classes = 3

model = mini_resnet_v2(input_shape,num_classes)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])


# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = Adam(lr=0.001)

input_shape = (32, 32, 3)
num_classes = 3

model = mini_resnet_v2(input_shape,num_classes)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])


# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

In [ ]:

from tensorflow.keras.optimizers import RMSprop
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer =  RMSprop(lr=0.001, rho=0.9, epsilon=1e-08)

model =MiniVGGNet.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

In [ ]:
imagePaths = list(paths.list_images("data2/output"))

# Bước 1. Chuẩn bị dữ liệu
# Khởi tạo tiền xử lý ảnh
sp = SimplePreprocessor(32, 32) # Thiết lập kích thước ảnh 32 x 32
iap = ImageToArrayPreprocessor() # Gọi hàm để chuyển ảnh sang mảng

# Nạp dataset từ đĩa
print("[INFO] Nạp ảnh...")

sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

[INFO] Nạp ảnh...
[INFO] Đã xử lý 500/25962
[INFO] Đã xử lý 1000/25962
[INFO] Đã xử lý 1500/25962
[INFO] Đã xử lý 2000/25962
[INFO] Đã xử lý 2500/25962
[INFO] Đã xử lý 3000/25962
[INFO] Đã xử lý 3500/25962
[INFO] Đã xử lý 4000/25962
[INFO] Đã xử lý 4500/25962
[INFO] Đã xử lý 5000/25962
[INFO] Đã xử lý 5500/25962
[INFO] Đã xử lý 6000/25962
[INFO] Đã xử lý 6500/25962
[INFO] Đã xử lý 7000/25962
[INFO] Đã xử lý 7500/25962
[INFO] Đã xử lý 8000/25962
[INFO] Đã xử lý 8500/25962
[INFO] Đã xử lý 9000/25962
[INFO] Đã xử lý 9500/25962
[INFO] Đã xử lý 10000/25962
[INFO] Đã xử lý 10500/25962
[INFO] Đã xử lý 11000/25962
[INFO] Đã xử lý 11500/25962
[INFO] Đã xử lý 12000/25962
[INFO] Đã xử lý 12500/25962
[INFO] Đã xử lý 13000/25962
[INFO] Đã xử lý 13500/25962
[INFO] Đã xử lý 14000/25962
[INFO] Đã xử lý 14500/25962
[INFO] Đã xử lý 15000/25962
[INFO] Đã xử lý 15500/25962
[INFO] Đã xử lý 16000/25962
[INFO] Đã xử lý 16500/25962
[INFO] Đã xử lý 17000/25962
[INFO] Đã xử lý 17500/25962
[INFO] Đã xử lý 18000/

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model =MiniVGGNet.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
305/305 [==============================] - 17s 16ms/step - loss: 0.9653 - accuracy: 0.5868 - val_loss: 1.2325 - val_accuracy: 0.4069
Epoch 2/50
305/305 [==============================] - 4s 12ms/step - loss: 0.7879 - accuracy: 0.6432 - val_loss: 0.9192 - val_accuracy: 0.6377
Epoch 3/50
305/305 [==============================] - 4s 12ms/step - loss: 0.7044 - accuracy: 0.6844 - val_loss: 0.6287 - val_accuracy: 0.6923
Epoch 4/50
305/305 [==============================] - 4s 12ms/step - loss: 0.6427 - accuracy: 0.7073 - val_loss: 0.8231 - val_accuracy: 0.6358
Epoch 5/50
305/305 [==============================] - 4s 12ms/step - loss: 0.5915 - accuracy: 0.7294 - val_loss: 0.5636 - val_accuracy: 0.7381
Epoch 6/50
305/305 [==============================] - 3s 11ms/step - loss: 0.5541 - accuracy: 0.7535 - val_loss: 0.5115 - val_accuracy: 0.7611
Epoch 7/50
305/305 [==============================] - 3s 11ms/step - loss: 0.5224 - ac

In [ ]:
model.save("modelver20.h5")

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

input_shape = (32, 32, 3)
num_classes = 3

model = VGG16(input_shape,num_classes)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
305/305 [==============================] - 26s 44ms/step - loss: 1.0988 - accuracy: 0.3409 - val_loss: 1.0991 - val_accuracy: 0.3520
Epoch 2/50
305/305 [==============================] - 12s 38ms/step - loss: 1.0932 - accuracy: 0.3597 - val_loss: 1.0466 - val_accuracy: 0.4485
Epoch 3/50
305/305 [==============================] - 12s 39ms/step - loss: 0.9547 - accuracy: 0.4925 - val_loss: 0.8074 - val_accuracy: 0.5685
Epoch 4/50
305/305 [==============================] - 12s 39ms/step - loss: 0.7790 - accuracy: 0.5802 - val_loss: 0.8252 - val_accuracy: 0.5538
Epoch 5/50
305/305 [==============================] - 12s 40ms/step - loss: 0.6963 - accuracy: 0.6177 - val_loss: 0.6587 - val_accuracy: 0.6238
Epoch 6/50
305/305 [==============================] - 12s 39ms/step - loss: 0.6486 - accuracy: 0.6386 - val_loss: 0.6366 - val_accuracy: 0.6494
Epoch 7/50
305/305 [==============================] - 12s 40ms/step - loss: 0.607

In [ ]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class MiniVGGNets:
    @staticmethod
    def build(width, height, depth, classes):
        # Khởi tạo mô hình, shape ảnh đầu vào và số kênh của ảnh đầu vào
        model = Sequential()
        input_shape = (height, width, depth)
        channel_dim = -1  # chỉ số của số kênh ảnh đầu vào
                          # giá trị -1 ý muốn nói chỉ số kênh nằm cuối cùng
                          # của danh sách chứa dữ liệu ảnh đầu vào

        # sử dụng 'channels_first' để cập nhật shape và số kênh ảnh đầu vào
        if K.image_data_format() == 'channels_first':
            input_shape = (depth, height, width)
            channel_dim = 1

        # Chuỗi layer đầu tiên  CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(32, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Chuỗi layer thứ hai CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Chuỗi layer thứ ba CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Thiết lập FC thứ nhất => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))   # Dropout 50%

        # THiết lập FC thứ hai => Hàm phân lớp Softmax
        model.add(Dense(classes))
        model.add(Activation('softmax'))

        # Trả về kiến trúc mạng/mô hình
        return model


In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model =MiniVGGNets.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=50, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/50
305/305 [==============================] - 20s 17ms/step - loss: 0.9642 - accuracy: 0.5987 - val_loss: 1.2091 - val_accuracy: 0.4981
Epoch 2/50
305/305 [==============================] - 5s 16ms/step - loss: 0.7199 - accuracy: 0.6772 - val_loss: 0.8159 - val_accuracy: 0.6320
Epoch 3/50
305/305 [==============================] - 5s 17ms/step - loss: 0.6525 - accuracy: 0.7037 - val_loss: 1.0113 - val_accuracy: 0.6427
Epoch 4/50
305/305 [==============================] - 5s 15ms/step - loss: 0.5900 - accuracy: 0.7329 - val_loss: 0.8130 - val_accuracy: 0.6777
Epoch 5/50
305/305 [==============================] - 5s 16ms/step - loss: 0.5459 - accuracy: 0.7502 - val_loss: 0.5042 - val_accuracy: 0.7652
Epoch 6/50
305/305 [==============================] - 5s 16ms/step - loss: 0.5036 - accuracy: 0.7734 - val_loss: 0.5488 - val_accuracy: 0.7618
Epoch 7/50
305/305 [==============================] - 4s 14ms/step - loss: 0.4576 - ac

In [ ]:

from keras.optimizers import Adam
# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Khởi tạo danh sách các label cho tập dữ liệu flowers
label_names = ["cat", "dog", "panda"]

# Bước 2: Khởi tạo bộ tối ưu và model
print("[INFO]: Biên dịch model....")
# Các tham số bộ tối ưu:
#   - learning_rate: Tốc dộ học
#   - decay: sử dụng để giảm từ từ tốc độ học theo thời gian
#            được tính bằng Tốc độ học /tổng epoch. Dùng để tránh overfitting
#            và tăng độ chính xác khi tranning
#   - momentum: Hệ số quán tính
#   - nesterov = True: sử dụng phương pháp tối ưu Nestrov accelerated gradient
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model =MiniVGGNets.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Bước 3: Train the network
print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=60, verbose=1)


model.summary() # Hiển thị tóm tắt các tham số của model

# Bước 4: Đánh giá mạng
print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=label_names))

[INFO]: Biên dịch model....
[INFO]: Đang trainning....
Epoch 1/60
305/305 [==============================] - 8s 17ms/step - loss: 0.9233 - accuracy: 0.6107 - val_loss: 1.1453 - val_accuracy: 0.5223
Epoch 2/60
305/305 [==============================] - 5s 17ms/step - loss: 0.7307 - accuracy: 0.6700 - val_loss: 1.8330 - val_accuracy: 0.4229
Epoch 3/60
305/305 [==============================] - 4s 15ms/step - loss: 0.6413 - accuracy: 0.7093 - val_loss: 0.8076 - val_accuracy: 0.6366
Epoch 4/60
305/305 [==============================] - 4s 15ms/step - loss: 0.5955 - accuracy: 0.7322 - val_loss: 0.6470 - val_accuracy: 0.7233
Epoch 5/60
305/305 [==============================] - 5s 17ms/step - loss: 0.5363 - accuracy: 0.7604 - val_loss: 0.5951 - val_accuracy: 0.7366
Epoch 6/60
305/305 [==============================] - 4s 15ms/step - loss: 0.5092 - accuracy: 0.7745 - val_loss: 0.8896 - val_accuracy: 0.6515
Epoch 7/60
305/305 [==============================] - 4s 15ms/step - loss: 0.4616 - acc